In [1]:
import os
os.getcwd()

'C:\\Users\\andrehadianto\\Documents\\CodingProjects\\50.040-NLP-Sentiment-Analysis\\data'

In [18]:
!python bert-ner_run.py --no_cuda --data_dir=partial/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_ner --max_seq_length=128 --do_train --num_train_epochs 3 --warmup_proportion=0.1

08/18/2020 18:01:57 - INFO - __main__ -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
08/18/2020 18:01:58 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at C:\Users\andrehadianto\.cache\torch\pytorch_transformers\5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
08/18/2020 18:01:59 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at C:\Users\andrehadianto\.cache\torch\pytorch_transformers\b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
08/18/2020 18:01:59 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attent

In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andrehadianto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
from bert import Ner
model = Ner('out_ner/')

text = 'British forces, based in the mainly south, have suffered far fewer losses than the much larger U.S. force fighting Sunni Arab insurgents and foreign fighters in the rest of Iraq.'
print("Text to predict Entity:", text)

output = model.predict(text)
for prediction in output:
    print(prediction)

Text to predict Entity: British forces, based in the mainly south, have suffered far fewer losses than the much larger U.S. force fighting Sunni Arab insurgents and foreign fighters in the rest of Iraq.
{'word': 'British', 'tag': 'B-gpe', 'confidence': 0.5998727679252625}
{'word': 'forces', 'tag': 'O', 'confidence': 0.9960131645202637}
{'word': ',', 'tag': 'O', 'confidence': 0.9965710639953613}
{'word': 'based', 'tag': 'O', 'confidence': 0.996905505657196}
{'word': 'in', 'tag': 'O', 'confidence': 0.9965936541557312}
{'word': 'the', 'tag': 'O', 'confidence': 0.9960546493530273}
{'word': 'mainly', 'tag': 'O', 'confidence': 0.9910088181495667}
{'word': 'south', 'tag': 'O', 'confidence': 0.9807453155517578}
{'word': ',', 'tag': 'O', 'confidence': 0.9971997737884521}
{'word': 'have', 'tag': 'O', 'confidence': 0.9975717663764954}
{'word': 'suffered', 'tag': 'O', 'confidence': 0.9974257349967957}
{'word': 'far', 'tag': 'O', 'confidence': 0.9960442781448364}
{'word': 'fewer', 'tag': 'O', 'conf

In [24]:
dataset = "partial"

In [25]:
sentences = list()
with open(os.path.join(dataset, "dev.in")) as f:
        lines = f.readlines()
        sentence = list()
        sent = ""
        for line in lines:
            formatted_line = line.strip()
            if(len(formatted_line) ==0):
                sentences.append(sent.strip())
                sent = ""
                continue
            sent += formatted_line+" "    
            
output_filename='dev.p6.model.out'
with open(os.path.join(dataset, output_filename), "w") as wf:
    for sentence in sentences:
        output = model.predict(sentence)
        for prediction in output:
            wf.write(prediction['word'] + " " + prediction['tag'] + "\n")               
        wf.write("\n")

In [27]:
from conlleval2 import evaluate, evaluate_conll_file

def eval(pred,gold):
    f_pred = open(pred,encoding = 'utf-8')
    f_gold = open(gold,encoding = 'utf-8')
    data_pred = f_pred.readlines()
    data_gold = f_gold.readlines()
    gold_tags = list()
    pred_tags = list()
    print(data_pred[0])
    for sentence in range(len(data_pred)):
        words_pred = data_pred[sentence].strip().split(' ')    
        words_gold = data_gold[sentence].strip().split(' ')  
        if len(words_gold)==1:
            continue
        gold_tags.append(words_gold[1])
        pred_tags.append(words_pred[1])
    return gold_tags,pred_tags

g_tags, p_tags = eval(os.path.join(dataset, 'dev.p6.model.out'), os.path.join(dataset, 'dev.out'))
print(evaluate(g_tags,p_tags,verbose=True))

The O

processed 2097 tokens with 236 phrases; found: 264 phrases; correct: 146.
accuracy:  63.24%; (non-O)
accuracy:  93.04%; precision:  55.30%; recall:  61.86%; FB1:  58.40
              art: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              eve: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              geo: precision:  58.82%; recall:  70.59%; FB1:  64.17  102
              gpe: precision:  81.82%; recall:  72.00%; FB1:  76.60  22
              nat: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              org: precision:  27.66%; recall:  37.14%; FB1:  31.71  47
              per: precision:  63.33%; recall:  59.38%; FB1:  61.29  30
              tim: precision:  57.14%; recall:  67.92%; FB1:  62.07  63
(55.3030303030303, 61.86440677966102, 58.400000000000006)
